In [2]:
#import something
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
%matplotlib inline

Load the data!

In [49]:
df_train=pd.read_csv('./data/train.csv')
df_test=pd.read_csv('./data/test.csv') 
df_train_test=pd.concat([df_train,df_test],axis=0)
# df_train_test=pd.get_dummies(df_train_test)
df_train=df_train_test[:len(df_train)]
df_test=df_train_test[len(df_train):]
df_train_without_price=df_train.drop('SalePrice',1)
print(df_train_without_price.shape)

(1460, 80)


Here I just use the PCA to reduce feature dimensions

In [115]:
col=~(df_train_without_price.applymap(np.isreal).all(0))
col_str=col.index.values[col.values]
col_val=col.index.values[~col.values]
df_train_str_col=pd.DataFrame(df_train_without_price,columns=col_str)
# df_train_str_col=df_train_str_col.fillna('Nan')

df_train_val_col=pd.DataFrame(df_train_without_price,columns=col_val)
df_train_str_col_dummy=pd.get_dummies(df_train_str_col)
price=df_train['SalePrice']
print(df_train_val_col.shape,df_train_str_col.shape,df_train_str_col_dummy.shape)


(1460, 37) (1460, 43) (1460, 252)


In [8]:
#pca=PCA(df_train_without_price.fillna(0).values)
pca = PCA(n_components=289)
pca.fit(df_train_without_price.fillna(0))
# print(pca.explained_variance_ratio_)

PCA(copy=True, iterated_power='auto', n_components=289, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

As we can tell from PCA projects into 289 different dimensions, the first 7th 
new projections' variances are at least in rank 10^(-3)
the first 12th new projections' variances are at least in rank 10^(-4).

In [11]:
# pca.fracs
price=df_train['SalePrice']
#need to move the price when calc pca
#pick the first 10 feature
T_df_tran_without_price=pd.DataFrame(pca.transform(df_train_without_price.fillna(0))[:,:10])

In [12]:
data=pd.concat([price,T_df_tran_without_price],axis=1)
data=data.dropna();

a simple regressor

In [13]:
from sklearn.ensemble import RandomForestRegressor

In [14]:
regr = RandomForestRegressor(max_depth=5, random_state=0)

In [15]:
regr.fit(data.drop('SalePrice',axis=1).values,data['SalePrice'].values )

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

also need to drop the NaN
I don't know why there will be NaN, shouldn't appear
need to fix this

In [19]:
df_test_without_price=df_test.drop('SalePrice',1)
T_df_test_without_price=pd.DataFrame(pca.transform(df_test_without_price.fillna(0))[:,:10])
id_T=pd.concat([df_test['Id'],T_df_test_without_price],axis=1)
data=id_T.dropna();

In [20]:
predict=regr.predict(data.drop('Id',axis=1).values)

write the upload result to result.csv

In [21]:
res=pd.DataFrame({'Id':data['Id'].values,'SalePrice':predict})
res.to_csv('./data/result.csv',index=False)